# Set Up

In [16]:
import pandas as pd
import altair as alt

In [17]:
pd.set_option('display.max_columns', None)

# Load Data

In [18]:
stats_df = pd.read_csv('player_stats_2025.csv')

# Visualizations

In [19]:
teams_of_interest = ['LAL', 'MIN']
teams_df = stats_df[stats_df['team'].isin(teams_of_interest)]

In [20]:
teams_df.head()

,id,playerName,position,age,games,gamesStarted,totalMinutes,fieldGoals,fieldAttempts,fieldPercent,threeFg,threeAttempts,threePercent,twoFg,twoAttempts,twoPercent,effectFgPercent,ft,ftAttempts,ftPercent,offensiveRb,defensiveRb,totalRb,assists,steals,blocks,turnovers,personalFouls,points,team,season,playerId,minutesPlayed,per,tsPercent,threePAR,ftr,offensiveRBPercent,defensiveRBPercent,totalRBPercent,assistPercent,stealPercent,blockPercent,turnoverPercent,usagePercent,offensiveWS,defensiveWS,winShares,winSharesPer,offensiveBox,defensiveBox,box,vorp,mpg,ppg,rpg,apg,bpg,spg
5,18860,Nickeil Alexander-Walker,SG,26,82,10,2073.0,270,616,0.438,141,370,0.381,129,246,0.524,0.553,92,118,0.780,54,211,265,223,50,34,99,139,773,MIN,2025,alexani01,2073,11.6,0.579,0.601,0.192,2.9,11.1,7.1,15.0,1.2,1.5,12.9,16.0,2.0,2.3,4.3,0.099,-0.8,0.4,-0.4,0.8,25.3,9.4,3.2,2.7,0.4,0.6
14,19143,Armel Traoré,SF,22,9,0,67.0,6,19,0.316,0,7,0.000,6,12,0.500,0.316,2,7,0.286,2,13,15,1,4,2,3,6,14,LAL,2025,armeltr01,67,3.5,0.317,0.368,0.368,3.5,21.2,12.6,2.0,2.9,2.8,12.0,16.4,-0.2,0.1,-0.1,-0.092,-9.5,-0.4,-9.8,-0.1,7.4,1.6,1.7,0.1,0.2,0.4
92,19272,Jaylen Clark,SG,23,40,4,522.0,56,120,0.467,22,51,0.431,34,69,0.493,0.558,29,37,0.784,26,27,53,26,36,2,8,61,163,MIN,2025,clarkja02,522,12.0,0.598,0.425,0.308,5.6,5.6,5.6,6.7,3.4,0.4,5.5,12.0,0.9,0.7,1.6,0.147,-2.9,2.7,-0.2,0.2,13.0,4.1,1.3,0.6,0.0,0.9
102,18906,Mike Conley,PG,37,71,64,1756.0,187,468,0.400,127,310,0.410,60,158,0.380,0.535,81,90,0.900,32,150,182,319,79,14,75,111,582,MIN,2025,conlemi01,1756,13.3,0.573,0.662,0.192,2.1,9.3,5.7,24.5,2.2,0.7,12.9,14.4,3.0,2.2,5.2,0.142,-0.1,1.3,1.2,1.4,24.7,8.2,2.6,4.5,0.2,1.1
125,19066,Rob Dillingham,PG,20,49,1,516.0,93,211,0.441,25,74,0.338,68,137,0.496,0.500,8,15,0.533,12,38,50,98,20,1,53,37,219,MIN,2025,dilliro01,516,10.2,0.503,0.351,0.071,2.6,8.0,5.4,28.4,1.9,0.2,19.6,22.7,-0.4,0.6,0.1,0.012,-3.3,-0.9,-4.2,-0.3,10.5,4.5,1.0,2.0,0.0,0.4


## Insight Into Player Performance

In [21]:
# Interactive scatter plot
scatter = alt.Chart(teams_df).mark_circle().encode(
    x=alt.X('per:Q', title='Player Efficiency Rating (PER)'),
    y=alt.Y('ppg:Q', title='Points Per Game'),
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    size=alt.Size('winShares:Q', title='Win Shares'),
    tooltip=['playerName', 'team', 'mpg', 'ppg', 'per', 'tsPercent', 'winShares']
).properties(
    title='Player Performance: Lakers vs Timberwolves',
    width=600,
    height=400
).interactive()

scatter

alt.Chart(...)

## Team Performance Comparison

In [22]:
# Aggregate team-level stats
team_stats = teams_df.groupby('team').agg(
    total_points=('points', 'sum'),
    total_field_goals=('fieldGoals', 'sum'),
    total_three_pointers=('threeFg', 'sum'),
    total_field_attempts=('fieldAttempts', 'sum'),
    total_rebounds=('totalRb', 'sum'),
    total_assists=('assists', 'sum'),
    total_games=('games', 'sum')
).reset_index()

# Calculate additional team-level metrics
team_stats['field_goal_percentage'] = team_stats['total_field_goals'] / team_stats['total_field_attempts'] * 100
team_stats['points_per_game'] = team_stats['total_points'] / team_stats['total_games']
team_stats['rebounds_per_game'] = team_stats['total_rebounds'] / team_stats['total_games']
team_stats['assists_per_game'] = team_stats['total_assists'] / team_stats['total_games']
team_stats['three_pointers_per_game'] = team_stats['total_three_pointers'] / team_stats['total_games']



# Create separate dataframes for each stat
ppg_chart_data = team_stats[['team', 'points_per_game']]
rpg_chart_data = team_stats[['team', 'rebounds_per_game']]
apg_chart_data = team_stats[['team', 'assists_per_game']]
fg_percent_chart_data = team_stats[['team', 'field_goal_percentage']]
three_pointers_chart_data = team_stats[['team', 'three_pointers_per_game']]


ppg_chart = alt.Chart(ppg_chart_data).mark_bar().encode(
    x='team:N',
    y='points_per_game:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team', 'points_per_game']
).properties(
    title="Points Per Game"
)

rpg_chart = alt.Chart(rpg_chart_data).mark_bar().encode(
    x='team:N',
    y='rebounds_per_game:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team', 'rebounds_per_game']
).properties(
    title="Rebounds Per Game"
)

apg_chart = alt.Chart(apg_chart_data).mark_bar().encode(
    x='team:N',
    y='assists_per_game:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team', 'assists_per_game']
).properties(
    title="Assists Per Game"
)

tpg_chart = alt.Chart(three_pointers_chart_data).mark_bar().encode(
    x='team:N',
    y='three_pointers_per_game:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team', 'three_pointers_per_game']
).properties(
    title="3PT Per Game"
)

fg_percent_chart = alt.Chart(fg_percent_chart_data).mark_bar().encode(
    x='team:N',
    y='field_goal_percentage:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team', 'field_goal_percentage']
).properties(
    title="Field Goal Percentage: Lakers vs Timberwolves"
)



# Combine the three charts into a single view
final_chart = alt.hconcat(
    ppg_chart,
    rpg_chart,
    apg_chart,
    tpg_chart,
    fg_percent_chart
)

final_chart

alt.HConcatChart(...)

## Player Position Analysis

In [23]:
position_stats = teams_df.groupby(['team', 'position']).agg(
    avg_ppg=('ppg', 'mean'),
    avg_rpg=('rpg', 'mean'),
    avg_apg=('apg', 'mean'),
    avg_bpg=('bpg', 'mean'),
    avg_spg=('spg', 'mean'),
).reset_index()

color_map = {'LAL': 'purple', 'MIN': 'skyblue'}

position_order = ['PG', 'SG', 'SF', 'PF', 'C']

ppg_position_chart = alt.Chart(position_stats).mark_bar().encode(
    x=alt.X('position:N', sort=position_order),
    y='avg_ppg:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team:N', 'position:N', 'avg_ppg:Q']
).properties(
    title='Average Points Per Game by Position'
)

rpg_position_chart = alt.Chart(position_stats).mark_bar().encode(
    x=alt.X('position:N', sort=position_order),
    y='avg_rpg:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team:N', 'position:N', 'avg_rpg:Q']
).properties(
    title='Average Rebounds Per Game by Position'
)

apg_position_chart = alt.Chart(position_stats).mark_bar().encode(
    x=alt.X('position:N', sort=position_order),
    y='avg_apg:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team:N', 'position:N', 'avg_apg:Q']
).properties(
    title='Average Assists Per Game by Position'
)

spg_position_chart = alt.Chart(position_stats).mark_bar().encode(
    x=alt.X('position:N', sort=position_order),
    y='avg_spg:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team:N', 'position:N', 'avg_spg:Q']
).properties(
    title='Average Steals Per Game by Position'
)

bpg_position_chart = alt.Chart(position_stats).mark_bar().encode(
    x=alt.X('position:N', sort=position_order),
    y='avg_bpg:Q',
    color=alt.Color('team:N', scale=alt.Scale(domain=['LAL', 'MIN'], range=['purple', 'skyblue'])),
    tooltip=['team:N', 'position:N', 'avg_bpg:Q']
).properties(
    title='Average Blocks Per Game by Position'
)



position_analysis_side_by_side = alt.hconcat(
    ppg_position_chart,
    rpg_position_chart,
    apg_position_chart,
    spg_position_chart,
    bpg_position_chart
)

position_analysis_side_by_side

alt.HConcatChart(...)